# Content:
## 1. Reading and Writing data in text format  
## 2. Binary Data Formats
## 3. Interacting with Web APIs

Data output and data input:
- text files or other more effient on-disk formats  
+ database  
* web APIs  

## 1. Reading and Writing data in text format  

***read_csv*** and ***read_table*** are the most used ones to convert data into a **Dataframe**  

Parsing functions:
>read_csv  
>read_table  
>read_excel  
>read_html  
>read_json  
>read_sql

In [49]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [22]:
df = pd.read_table('examples/ex1.csv',sep=',')
df
#also could use read_table and specified the delimiter

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


1.1 Assign **column names**:

In [18]:
pd.read_csv('examples/ex2.csv', header=None)
#default names

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [19]:
pd.read_csv('examples/ex2.csv', names=['a','b','c','d','message'])
#specify yourself

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


1.2 Columns to be the **index** of the returned DataFrame:

In [20]:
names=['a','b','c','d','message']
pd.read_csv('examples/ex2.csv',names=names, index_col='message')
#if u want a hierarchical index, pass a list of column numbers or names

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


1.3 Cases a table dont have a **fixed delimiter**:

In [23]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

1.4 Use **\s+** to sep a variable amount of whitespace

In [24]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


1.5 Using **skiprows** to skip rows:

In [25]:
pd.read_csv('examples/ex4.csv', skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


1.6 Handling **missing values**:

In [27]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


The **na_values** option cat either a list or set of strings ot consider missing values

In [29]:
sentinels = {'message':['foo', 'NA'], 'something':['two']}
pd.read_csv('examples/ex5.csv', na_values = sentinels)
#Different NA sentinels can be specified for each column in a dict

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in *Pieces*

1.7 When we want to read in **a small piece** of a file or **iterate** through smaller chunks of the file

In [59]:
result = pd.read_csv('examples/ex6.csv')
result.tail()

,one,two,three,four,key
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G
9999,-0.096376,-1.012999,-0.657431,-0.573315,0


Specify with **nrows**:

In [31]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


Specify a chunksize as a number of rows and returns a TextParser object, then u can **iterate** it:

In [44]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

In [45]:
tot = pd.Series([])

In [58]:
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:5]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
dtype: float64

### Writing Data to Text Format

1.8 Using **to_csv** method to write a comma-separated file by default:

In [60]:
import sys
data = pd.read_csv('examples/ex5.csv')

In [61]:
data.to_csv(sys.stdout)

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [64]:
data.to_csv(sys.stdout, sep='|', na_rep='Null', index=False, header=False, columns=['a','b','c'])
#sep to change the delimiter
#na_rep to replce the NA
#labels can be disabled
#columns to choose the columns u want to write(a subset)

1|2|3.0
5|6|Null
9|10|11.0


### Working with Delimited Formats

1.9 When file have one or more malformed lines that trip up **read_table**, use **import csv**:

In [65]:
import csv

In [69]:
pd.read_table('examples/ex7.csv')

,"a,""b"",""c"""
0,"1,""2"",""3"""
1,"1,""2"",""3"""


In [66]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
#read the file into a list of lines

In [70]:
for line in lines:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [71]:
header, values = lines[0], lines[1:]

In [78]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
#using dictionary comprehension and the expression zip(*values), which transposes rows to columns
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [74]:
pd.DataFrame(data_dict)

,a,b,c
0,1,2,3
1,1,2,3


### JSON Data

1.10  
JSON(Javascript Object Notation) has become one of the standard formats for sending data by HTTP request.  
    It has a much more free_form data.

In [10]:
import json

In [15]:
obj = """
{"names": "Wes",
 "pets": null,
 "places_lived": ["United States", "Spain"],
 "siblings": [{"name": "Scott","age":30},
              {"name": "Ketie", "age": 38}]
}               
"""

In [22]:
result = json.loads(obj)
#Convert a JSON string to Python form
asjson = json.dumps(result)
#Convert a Pyhton form to JSON
result

{'names': 'Wes',
 'pets': None,
 'places_lived': ['United States', 'Spain'],
 'siblings': [{'name': 'Scott', 'age': 30}, {'name': 'Ketie', 'age': 38}]}

In [23]:
siblings = pd.DataFrame(result['siblings'], columns = ['name', 'age'])
siblings

,name,age
0,Scott,30
1,Ketie,38


**pandas.read_json** can auto convert

In [24]:
data = pd.read_json('examples/example.json')
data
#data.to_json

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


### XML and HTML: Web Scraping

1.11  
The **pandas.read_html** function parse all tabular data contained within tabel tags by default and returns a **list** of Dataframe objects.

In [41]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')
tables[0].head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


## 2. Binary Data Formats

### Pickle

2.1  
**pickle** serializaition, **to_pickle** method and **pandas.read_pickle**

In [44]:
frame = pd.read_csv('examples/ex1.csv')
frame.to_pickle('examples/frame_pickle')

In [45]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**pickle** is only recommended as a short-term storage format

### HDF5

2.2  
*hierarchical data format* is a well-regarded file format intended for **large quantities of scientific array data**.  


The **HDFStore** class works like a dictt and handles the low-level details

In [51]:
store = pd.HDFStore('examples/mydata.h5')
# init a HDf5 file

In [52]:
frame = pd.DataFrame({'a':np.random.randn(100)})

In [56]:
store['obj1'] = frame

In [57]:
store['obj_col'] = frame['a']

In [58]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: examples/mydata.h5

In [79]:
store.open()

In [80]:
store.put('obj2', frame, format='table')
# or fomat = 'fixed', but ist can use syntax like below(where=['...'])

In [70]:
store.select('obj2', where=['index >=10 and index <= 15'])

,a
10,0.325813
11,1.840935
12,-1.480623
13,-0.496269
14,-2.452864
15,0.500816


In [72]:
store.close()

In [75]:
pd.read_hdf('examples/mydata.h5', 'obj2', where=['index < 5'])

,a
0,0.932158
1,-0.122401
2,-1.991665
3,-1.109094
4,0.911467


### Microsoft Exel Files

2.3  
**pandas_read_excel**

In [84]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


>to_excel(index=True)  

To write pandas data to Excel format, use **to_excel**, and u can expert without index by set **index=False**

In [85]:
frame.to_excel('examples/ex2.xlsx')

## 3. Interacting with Web APIs

many websites have public APIs providing data feeds via JSON or some other format.  
One easy-to-use method to access these APIs is the **requests package**

In [86]:
import requests

In [132]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

In [135]:
data = resp.json()

In [136]:
data[0]['title']

"Str.encode('base64') turns arrays of strings to NaN and fails on byte strings"

In [139]:
issues = pd.DataFrame(data, columns=['number', 'title', 'user', 'state'])
issues.head()

,number,title,user,state
0,32049,Str.encode('base64') turns arrays of strings t...,"{'login': 'bwanaaa', 'id': 4876720, 'node_id':...",open
1,32048,Pandas 1.0.1 gives 1953 TypeError: Cannot use ...,"{'login': 'bwanaaa', 'id': 4876720, 'node_id':...",open
2,32047,use numexpr for Series comparisons,"{'login': 'jbrockmendel', 'id': 8078968, 'node...",open
3,32046,Use fixtures in pandas/tests/base,"{'login': 'SaturnFromTitan', 'id': 17853006, '...",open
4,32045,Unable to Install Pandas on Python 3.9.0a3,"{'login': 'yogendrasoni79', 'id': 6441478, 'no...",open


## 4. Interacting with Databases

Loading data from SQL into a DataFrame is fairly straightforward by **pandas.read_sql** methods

In [128]:
import sqlalchemy as sqla

In [129]:
import pymysql
db = pymysql.connect(host='localhost',port =3306,user='root',passwd='123456',db='scholarship',charset='utf8' )

In [130]:
pd.read_sql( 'select * from stu_info', db).tail()

,stu_id,stu_name,stu_major,stu_class,card_num,stu_scholarship,stu_scholarship_status
15,19208052,张连明,计算机技术,190802,621712831101757101011106310,是,否
16,19208053,张文楷,计算机技术,190802,62172484137464156538905,是,否
17,19208054,张亚平,计算机技术,190802,62179646215216762917165,是,否
18,19208059,周珊琳,计算机技术,190802,6217685641064346451027076,是,否
19,19208060,周师严,计算机技术,190802,621797601090388512331766,是,否


In [131]:
db.close()